In [ ]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Define the folder path within Google Drive
drive_folder = '/content/drive/MyDrive/Project/'

In [1]:
import os

drive_folder = 'D:/Training/Software Office/Web Scraping'

In [2]:
import requests
import re
from datetime import datetime, timedelta
import time

# Define the ICAO code, start date, and end date for data retrieval
ICAO = 'widd'
back_date = datetime(2010, 1, 19)
now_date = datetime(2023, 8, 1)

# Calculate the time interval for data retrieval
delta = timedelta(days=1)
dates = []
while back_date <= now_date:
    dates.append(back_date.strftime("%d/%m/%Y"))
    back_date += delta

# Define the hours for data retrieval
fhour = ['0', '6', '12', '18']
uhour = ['5', '11', '17', '23']

# Define the cookies and headers for making requests
cookies = {
    'google-analytics_v4_0bd2__ga4': '0c2334cd-b22d-42a6-83d0-6704543c5f21',
    'google-analytics_v4_0bd2___z_ga_audiences': '0c2334cd-b22d-42a6-83d0-6704543c5f21',
    '_hjSessionUser_3310300': 'eyJpZCI6IjIwNmQ4MWExLThkMmYtNWZlMS04YjAzLTJhZGI3NjBmODM5MSIsImNyZWF0ZWQiOjE2OTE4Mjc1Mzg0NTcsImV4aXN0aW5nIjp0cnVlfQ==',
    '_ga': 'GA1.1.978655597.1691827538',
    'PHPSESSID': 'qovlvenaa1rqbhokdg221ehju1',
    '_ga_139PJPE1TR': 'GS1.1.1692978927.36.1.1692978936.0.0.0',
    'google-analytics_v4_0bd2__counter': '356',
    'google-analytics_v4_0bd2__ga4sid': '1356238444',
    'google-analytics_v4_0bd2__session_counter': '42',
    'google-analytics_v4_0bd2__let': '1693139433428',
    'google-analytics_v4_0bd2__engagementPaused': '1693139433428',
    'google-analytics_v4_0bd2__engagementStart': '1693139183650',
}

headers = {
    'authority': 'aviation.bmkg.go.id',
    'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
    'accept-language': 'en-US,en;q=0.9,tr;q=0.8',
    'cache-control': 'max-age=0',
    'content-type': 'application/x-www-form-urlencoded',
    'cookie': 'google-analytics_v4_0bd2__ga4=0c2334cd-b22d-42a6-83d0-6704543c5f21; google-analytics_v4_0bd2___z_ga_audiences=0c2334cd-b22d-42a6-83d0-6704543c5f21; _hjSessionUser_3310300=eyJpZCI6IjIwNmQ4MWExLThkMmYtNWZlMS04YjAzLTJhZGI3NjBmODM5MSIsImNyZWF0ZWQiOjE2OTE4Mjc1Mzg0NTcsImV4aXN0aW5nIjp0cnVlfQ==; _ga=GA1.1.978655597.1691827538; PHPSESSID=qovlvenaa1rqbhokdg221ehju1; _ga_139PJPE1TR=GS1.1.1692978927.36.1.1692978936.0.0.0; google-analytics_v4_0bd2__counter=356; google-analytics_v4_0bd2__ga4sid=1356238444; google-analytics_v4_0bd2__session_counter=42; google-analytics_v4_0bd2__let=1693139433428; google-analytics_v4_0bd2__engagementPaused=1693139433428; google-analytics_v4_0bd2__engagementStart=1693139183650',
    'origin': 'https://aviation.bmkg.go.id',
    'referer': 'https://aviation.bmkg.go.id/web/metar_speci.php?icao=widd&sa=yes&fd=19%2F01%2F2010&fh=0&fm=59&ud=19%2F01%2F2010&uh=6&um=59&f=raw_format',
    'sec-ch-ua': '"Chromium";v="116", "Not)A;Brand";v="24", "Google Chrome";v="116"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'document',
    'sec-fetch-mode': 'navigate',
    'sec-fetch-site': 'same-origin',
    'sec-fetch-user': '?1',
    'upgrade-insecure-requests': '1',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/116.0.0.0 Safari/537.36',
}

# Define the pattern for data extraction
pattern = r'<span class="data_text">(.*?)<\/span>'

# Define the path in your Google Drive
drive_filename = os.path.join(drive_folder, f"{ICAO}_all_data.txt")

# Define the file to store the last fetched date
last_fetched_date_file = os.path.join(drive_folder, "last_fetched_date.txt")

# Function to check if fetching should be paused
def should_pause():
    return os.path.exists(pause_file)

# Function to continue fetching
def continue_fetching():
    if os.path.exists(last_fetched_date_file):
        with open(last_fetched_date_file, 'r') as f:
            last_fetched_date = f.read().strip()
    else:
        last_fetched_date = None

    # Get the index of the last fetched date or 0 if not found
    start_date_index = dates.index(last_fetched_date) + 1 if last_fetched_date else 0

    # Open the consolidated file for writing in append mode
    with open(drive_filename, "a") as file:
        for date_index, date in enumerate(dates):
            if date_index < start_date_index:
                continue  # Skip dates before the start_date_index for resuming

            # Initialize a variable to store fetched data for the same date
            consolidated_data = ""

            for u, hour in enumerate(fhour):
                # Check if user requested to pause (use an appropriate condition)
                if should_pause():
                    print("Pausing the scraping process...")
                    while should_pause():
                        time.sleep(10)  # Adjust the time interval as needed

                params = {
                    'icao': ICAO,
                    'sa': 'yes',
                    'fd': date,
                    'fh': hour,
                    'fm': '0',
                    'ud': date,
                    'uh': uhour[u],
                    'um': '59',
                    'f': 'raw_format',
                }

                response = requests.get('https://aviation.bmkg.go.id/web/metar_speci.php', params=params, cookies=cookies, headers=headers)

                if response.status_code == 200:
                    response_text = response.content.decode('utf-8')

                    # Check if the pattern exists in the response_text
                    if re.search(pattern, response_text):
                        matches = re.findall(pattern, response_text)

                        for match in matches:
                            cleaned_data = re.sub(r'<br>', ' ', match)
                            parsed_date = datetime.strptime(date, "%d/%m/%Y").strftime("%d-%b-%Y")
                            consolidated_data += parsed_date + " " + cleaned_data + '\n'

                    else:
                        print(f"Pattern not found for date {date} and hour {hour}.")
                else:
                    print(f"Failed to fetch content for date {date} and hour {hour}.")

            # Write the consolidated data for the same date to the file
            file.write(consolidated_data)

            # Save the last fetched date to the file
            with open(last_fetched_date_file, 'w') as f:
                f.write(date)
                print(f"Fetch content for date {date}.")

# Define the pause file path
pause_file = os.path.join(drive_folder, "pause.txt")

In [ ]:
# Run web scraping or continue from the last fetched date
continue_fetching()

Fetch content for date 16/10/2010.
Pattern not found for date 17/10/2010 and hour 12.
Fetch content for date 17/10/2010.
Pattern not found for date 18/10/2010 and hour 18.
Fetch content for date 18/10/2010.
Pattern not found for date 19/10/2010 and hour 18.
Fetch content for date 19/10/2010.
Pattern not found for date 20/10/2010 and hour 18.
Fetch content for date 20/10/2010.
Fetch content for date 21/10/2010.
Pattern not found for date 22/10/2010 and hour 0.
Fetch content for date 22/10/2010.
Pattern not found for date 23/10/2010 and hour 6.
Fetch content for date 23/10/2010.
Pattern not found for date 24/10/2010 and hour 12.
Fetch content for date 24/10/2010.
Pattern not found for date 25/10/2010 and hour 18.
Fetch content for date 25/10/2010.
Pattern not found for date 26/10/2010 and hour 18.
Fetch content for date 26/10/2010.
Fetch content for date 27/10/2010.
Fetch content for date 28/10/2010.
Fetch content for date 29/10/2010.
Fetch content for date 30/10/2010.
Pattern not found 

Pattern not found for date 06/02/2011 and hour 0.
Pattern not found for date 06/02/2011 and hour 6.
Pattern not found for date 06/02/2011 and hour 12.
Fetch content for date 06/02/2011.
Pattern not found for date 07/02/2011 and hour 0.
Pattern not found for date 07/02/2011 and hour 6.
Pattern not found for date 07/02/2011 and hour 12.
Fetch content for date 07/02/2011.
Pattern not found for date 08/02/2011 and hour 12.
Pattern not found for date 08/02/2011 and hour 18.
Fetch content for date 08/02/2011.
Pattern not found for date 09/02/2011 and hour 12.
Pattern not found for date 09/02/2011 and hour 18.
Fetch content for date 09/02/2011.
Pattern not found for date 10/02/2011 and hour 18.
Fetch content for date 10/02/2011.
Fetch content for date 11/02/2011.
Fetch content for date 12/02/2011.
Fetch content for date 13/02/2011.
Pattern not found for date 14/02/2011 and hour 18.
Fetch content for date 14/02/2011.
Fetch content for date 15/02/2011.
Fetch content for date 16/02/2011.
Fetch c

Fetch content for date 28/06/2011.
Fetch content for date 29/06/2011.
Fetch content for date 30/06/2011.
Fetch content for date 01/07/2011.
Fetch content for date 02/07/2011.
Fetch content for date 03/07/2011.
Pattern not found for date 04/07/2011 and hour 18.
Fetch content for date 04/07/2011.
Pattern not found for date 05/07/2011 and hour 18.
Fetch content for date 05/07/2011.
Fetch content for date 06/07/2011.
Fetch content for date 07/07/2011.
Pattern not found for date 08/07/2011 and hour 18.
Fetch content for date 08/07/2011.
Fetch content for date 09/07/2011.
Pattern not found for date 10/07/2011 and hour 18.
Fetch content for date 10/07/2011.
Fetch content for date 11/07/2011.
Fetch content for date 12/07/2011.
Fetch content for date 13/07/2011.
Fetch content for date 14/07/2011.
Pattern not found for date 15/07/2011 and hour 18.
Fetch content for date 15/07/2011.
Fetch content for date 16/07/2011.
Fetch content for date 17/07/2011.
Fetch content for date 18/07/2011.
Fetch cont